In [29]:
from scipy.io import loadmat
from scipy.io import savemat
import pandas as pd
import numpy as np

mdata = loadmat('dataset/sample_wave.mat')['sample_wave']
signals = mdata['voluntary'][0,0]
epochs = signals.dtype.names

for epoch in epochs:
    fp1 = signals[epoch][0,0][0]
    fp2 = signals[epoch][0,0][1]
    print(fp1)

[10.44294637 12.39616953 14.85475781 ...  0.29588827 -0.51760931
 -0.6013602 ]
[ 1.62671141e+00  3.12707470e+00 -7.56913686e-04 ... -2.17007901e+00
 -2.63844709e+00 -6.31337254e+00]
[1.45594763 4.09428963 5.80348001 ... 1.2303676  2.22374942 2.02752176]
[ 3.54721036  5.65187592  6.8573502  ... -7.36770304 -6.47218374
 -4.4251172 ]
[ 0.45426849 -1.12495929 -2.20862893 ... -2.90535322 -4.78143809
 -5.52549105]
[-2.43177478 -3.89994877 -3.55620219 ... -0.77341509 -1.12917551
 -0.49535806]
[-1.58631863 -5.31708868 -5.6709105  ... -7.91676391 -9.59532296
 -9.58141407]
[-1.96880542 -0.84736994  0.56649245 ... -0.29551097  0.81768973
  1.66624367]
[-8.65797870e+00 -8.64263493e+00 -6.81672901e+00 ...  8.17310093e-03
  4.15233130e-01  9.48312777e-01]
[ 5.22457917  5.51334107  2.04782909 ... -8.07787906 -8.07530373
 -7.62325524]
[6.95464453 5.56535103 4.83091984 ... 3.94937505 4.28054211 2.89530534]
[5.91563228 3.81807099 2.94348934 ... 1.54481374 4.06111509 2.94651021]
[-5.54754471 -4.30641053 